# Convolution Image Net

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.nn_visualization import variable_summaries
from utils.data import init_model_logging
from utils.nn_graph import simple_layer
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import pandas as pd
import matplotlib.pylab as plt

## Load data

In [2]:
data = input_data.read_data_sets('/data/fashion/', one_hot=True)
img_shape = (28, 28)
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /data/fashion/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /data/fashion/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /data/fashion/t10k-images-idx3-ubyte.gz
Extracting /data/fashion/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Custom Layer Functions

In [ ]:
def conv_layer(name, input_data, 
               conv_filter_shape, pooling_filter_shape, 
               conv_filter_stride=[1, 1, 1, 1], 
               pooling_filter_stride=[1, 1, 1, 1],
               padding='SAME',
               is_training=True):
    w_name = 'w_' + name
    b_name = 'b_' + name
    
    # Variables initialization
    w = tf.get_variable(w_name, conv_filter_shape, initializer=tf.contrib.layers.variance_scaling_initializer())
    bias = tf.get_variable(b_name, initializer=tf.constant_initializer(0), shape=conv_filter_shape[-1])

    # Convolution part
    conv_layer = tf.nn.conv2d(input_data, w, strides=conv_filter_stride, padding=padding)
    # conv_layer = tf.contrib.layers.batch_norm(conv_layer, is_training=is_training)

    conv_layer = conv_layer + bias
    conv_layer = tf.nn.relu(conv_layer)

    # Pooling part
    conv_layer = tf.nn.max_pool(conv_layer, 
                                ksize=pooling_filter_shape, 
                                strides=pooling_filter_stride, 
                                padding=padding)
    
    return conv_layer

## Build Conv Net Graph

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('conv_image_net_inputs'):
        images = tf.placeholder(tf.float32, shape=[None, 784], name='images')
        labels = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
        keep_dropout_prob = tf.placeholder(tf.float32, name='keep_dropout_prob')
        is_training = tf.placeholder(tf.bool, name='is_training')
            
    with tf.name_scope('image_reshape'):        
        images_reshaped = tf.reshape(images, [-1, 28, 28, 1])

    with tf.variable_scope('conv_layer_1'):
        conv_layer_1 = conv_layer('cl_1', images_reshaped, 
               conv_filter_shape=[3, 3, 1, 32], pooling_filter_shape=[1, 2, 2, 1], 
               conv_filter_stride=[1, 1, 1, 1], pooling_filter_stride=[1, 2, 2, 1],
               padding='SAME')  

    with tf.variable_scope('conv_layer_2'):
        conv_layer_2 = conv_layer('cl_2', conv_layer_1, 
               conv_filter_shape=[3, 3, 32, 64], pooling_filter_shape=[1, 2, 2, 1], 
               conv_filter_stride=[1, 1, 1, 1], pooling_filter_stride=[1, 2, 2, 1],
               padding='SAME') 
    
    with tf.variable_scope('feed_forward_layer_1'):
        ff_layer_1 = tf.reshape(conv_layer_2, [-1, 7*7*64])
        ff_layer_1 = simple_layer('ff_1', ff_layer_1, shape=[7*7*64, 10], activation='linear')
        raw_prediction = tf.nn.dropout(ff_layer_1, keep_dropout_prob)
    
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(raw_prediction)

    with tf.name_scope('loss'):
        cross_entropy_vector = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=raw_prediction)
        loss = tf.reduce_mean(cross_entropy_vector)
        variable_summaries('loss_summary', cross_entropy_vector)
        
    with tf.name_scope('training'):
        # This dependency is here because of potential batch normalization and need of floating averages update
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_step  = tf.train.AdamOptimizer(1e-3).minimize(loss)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        variable_summaries('accuracy_summary', correct_prediction)
    
    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

## Init Model Logging

In [ ]:
base_dir = '/tensorboard_summaries/conv_image_net/'
logging_meta = init_model_logging(base_dir, 'experiment_final', graph=graph, remove_existing=True)

## Run Conv Net

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']

validation_feed_dict = {
    images: data.validation.images, 
    labels: data.validation.labels,
    keep_dropout_prob: 1., 
    is_training: False}

session = tf.Session(graph=graph, config=config)
session.run(initialize_vars)

for iteration in range(101):
    ##################
    # Training Phase #
    ##################
    
    _images, _labels = data.train.next_batch(100)
    feed_dict = {images: _images, labels: _labels, keep_dropout_prob: 0.5, is_training: True}
    _ = session.run([train_step], feed_dict=feed_dict)
 

    #################
    # Logging Phase #
    #################

    # Train log
    feed_dict={images: _images, labels: _labels, keep_dropout_prob: 1., is_training: False}
    _summary, _accuracy, _loss = session.run([merge_summaries, accuracy, loss],feed_dict=feed_dict)
    logging_meta['train_writer'].add_summary(_summary, iteration)

    # Valid Log
    if iteration % 100 == 0:
        _summary, _accuracy, _loss = session.run([merge_summaries, accuracy, loss], validation_feed_dict)
        
        logging_meta['valid_writer'].add_summary(_summary, iteration)
        logging_meta['saver'].save(session, model_path, iteration)
        print("= Valid Iteration {}: loss {}, accuracy {} =".format(iteration, _loss, _accuracy))

_prediction, = session.run([prediction], feed_dict=validation_feed_dict)

In [ ]:
session.close()

## Keras Part

In [ ]:
import keras as k
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense
from keras.models import Model

In [ ]:
images = Input(shape=(28, 28, 1))
layer = Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(images)
layer = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(layer)

layer = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='relu')(images)
layer = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')(layer)

layer = Flatten()(layer)
prediction = Dense(10, activation='softmax')(layer)

model = Model(inputs=images, outputs=prediction)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
images = data.train.images.reshape((-1, 28, 28, 1))
labels = data.train.labels

In [ ]:
valid_images = data.validation.images.reshape((-1, 28, 28, 1))
valid_labels = data.validation.labels

In [ ]:
model.fit(x=images, y=labels, batch_size=32, epochs=5, 
          validation_data=(valid_images, valid_labels))

In [ ]:
_prediction = model.predict(valid_images)

## PyTorch

In [7]:
import torch

In [49]:
dtype = torch.float
device = torch.device("cpu")

N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
loss_fn = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.Adam([w1, w2], lr=learning_rate)


In [51]:
for t in range(100000):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = loss_fn(y_pred, y)
    if t%1000 == 0:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    

0 33174540.0
1000 32282524.0
2000 31407946.0
3000 30551264.0
4000 29709850.0
5000 28885426.0
6000 28075776.0
7000 27281814.0
8000 26502998.0
9000 25738418.0
10000 24988076.0
11000 24251710.0
12000 23529110.0
13000 22820762.0
14000 22126830.0
15000 21446944.0
16000 20781058.0
17000 20128876.0
18000 19490962.0
19000 18866906.0
20000 18256076.0
21000 17658252.0
22000 17073316.0
23000 16501061.0
24000 15941383.0
25000 15394809.0
26000 14861603.0
27000 14340860.0
28000 13832507.0
29000 13336252.0
30000 12852342.0
31000 12380270.0
32000 11919481.0
33000 11469913.0
34000 11031235.0
35000 10603697.0
36000 10187285.0
37000 9781710.0
38000 9386925.0
39000 9002984.0
40000 8630072.0
41000 8268239.5
42000 7916777.5
43000 7575345.0
44000 7244323.5
45000 6923452.5
46000 6612356.5
47000 6310855.0
48000 6019002.0
49000 5736856.5
50000 5463679.0
51000 5200102.5
52000 4945442.0
53000 4699418.5
54000 4461374.5
55000 4231956.0
56000 4010873.5
57000 3797960.25
58000 3592799.25
59000 3395201.0
60000 3205204.

In [56]:
w1.grad

tensor([[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]])

## Results evaluation

In [ ]:
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_accuracy
from utils.results_evaluation import get_false_positives
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_rec_prec
from utils.results_evaluation import plot_coocurance_matrix
from utils.results_evaluation import plot_examples 

In [ ]:
df = get_info_df(data.validation.labels, _prediction, class_id2class_name_mapping, data.validation.images)

In [ ]:
get_accuracy(df)

In [ ]:
get_rec_prec(df, class_id2class_name_mapping)

In [ ]:
fp = get_false_positives(df, 'Shirt')

In [ ]:
plot_examples(fp)

In [ ]:
plot_coocurance_matrix(df, use_top3=False, use_log=False)